<a href="https://colab.research.google.com/github/AbimaelLopez/Diplomado/blob/main/Covid19Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
import numpy as np


url = "/content/COVID19MEXICO.csv"

covid_data = pd.read_csv(url)


print(covid_data.describe())

<ipython-input-26-a86c4a803dfc>:15: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_data = pd.read_csv(url)


             ORIGEN        SECTOR    ENTIDAD_UM          SEXO   ENTIDAD_NAC  \
count  1.267606e+06  1.267606e+06  1.267606e+06  1.267606e+06  1.267606e+06   
mean   1.648311e+00  7.111490e+00  1.462129e+01  1.412794e+00  1.542067e+01   
std    4.774977e-01  3.846018e+00  7.830616e+00  4.923366e-01  9.542388e+00   
min    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
25%    1.000000e+00  4.000000e+00  9.000000e+00  1.000000e+00  9.000000e+00   
50%    2.000000e+00  4.000000e+00  1.300000e+01  1.000000e+00  1.400000e+01   
75%    2.000000e+00  1.200000e+01  2.100000e+01  2.000000e+00  2.100000e+01   
max    2.000000e+00  9.900000e+01  3.200000e+01  2.000000e+00  9.900000e+01   

        ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE      INTUBADO      NEUMONIA  \
count  1.267606e+06   1.267606e+06   1.267606e+06  1.267606e+06  1.267606e+06   
mean   1.480011e+01   3.865859e+01   1.101725e+00  8.734962e+01  2.399891e+00   
std    7.792611e+00   7.368068e+01   3.022862

In [27]:
for col in covid_data.columns:
    print(col)

FECHA_ACTUALIZACION
ID_REGISTRO
ORIGEN
SECTOR
ENTIDAD_UM
SEXO
ENTIDAD_NAC
ENTIDAD_RES
MUNICIPIO_RES
TIPO_PACIENTE
FECHA_INGRESO
FECHA_SINTOMAS
FECHA_DEF
INTUBADO
NEUMONIA
EDAD
NACIONALIDAD
EMBARAZO
HABLA_LENGUA_INDIG
INDIGENA
DIABETES
EPOC
ASMA
INMUSUPR
HIPERTENSION
OTRA_COM
CARDIOVASCULAR
OBESIDAD
RENAL_CRONICA
TABAQUISMO
OTRO_CASO
TOMA_MUESTRA_LAB
RESULTADO_LAB
TOMA_MUESTRA_ANTIGENO
RESULTADO_ANTIGENO
CLASIFICACION_FINAL
MIGRANTE
PAIS_NACIONALIDAD
PAIS_ORIGEN
UCI


In [28]:
covid_data_t = covid_data.drop(columns=['FECHA_ACTUALIZACION', 'ID_REGISTRO','ORIGEN','SECTOR','ENTIDAD_UM','SEXO','ENTIDAD_NAC','TIPO_PACIENTE','FECHA_INGRESO','FECHA_DEF','INTUBADO','NEUMONIA','EDAD','NACIONALIDAD','EMBARAZO','HABLA_LENGUA_INDIG','INDIGENA','DIABETES','EPOC','ASMA','INMUSUPR','HIPERTENSION','OTRA_COM','CARDIOVASCULAR','OBESIDAD','RENAL_CRONICA','TABAQUISMO','OTRO_CASO','TOMA_MUESTRA_LAB','RESULTADO_LAB','TOMA_MUESTRA_ANTIGENO','RESULTADO_ANTIGENO','MIGRANTE','PAIS_NACIONALIDAD','PAIS_ORIGEN','UCI'])
for col in covid_data_t.columns:
    print(col)

covid_data_t = covid_data_t[((covid_data_t.CLASIFICACION_FINAL == 1) | (covid_data_t.CLASIFICACION_FINAL == 2) | (covid_data_t.CLASIFICACION_FINAL==3)) & (covid_data_t.ENTIDAD_RES == 9)]

covid_data_t = covid_data_t.drop(columns=['CLASIFICACION_FINAL','ENTIDAD_RES'])

print(covid_data_t.head)

print(covid_data_t.describe())


ENTIDAD_RES
MUNICIPIO_RES
FECHA_SINTOMAS
CLASIFICACION_FINAL
<bound method NDFrame.head of          MUNICIPIO_RES FECHA_SINTOMAS
24                   5     2023-01-30
29                  13     2023-01-08
30                  13     2023-01-02
50                  16     2023-05-08
74                  15     2023-02-07
...                ...            ...
1265533              7     2023-03-06
1266527              4     2023-05-15
1266682             12     2023-05-25
1266841              4     2023-06-01
1267596             16     2023-08-29

[116825 rows x 2 columns]>
       MUNICIPIO_RES
count  116825.000000
mean        8.975065
std         5.311545
min         2.000000
25%         5.000000
50%         8.000000
75%        12.000000
max       999.000000


In [29]:
covid_data_daily = covid_data_t.groupby(by = ['FECHA_SINTOMAS','MUNICIPIO_RES'])['MUNICIPIO_RES'].count().to_frame(name='CANTIDAD').reset_index()

print(covid_data_daily.head)

print(covid_data_daily.describe())

for col in covid_data_daily.columns:
    print(col)

<bound method NDFrame.head of      FECHA_SINTOMAS  MUNICIPIO_RES  CANTIDAD
0        2023-01-01              2        82
1        2023-01-01              3       102
2        2023-01-01              4        13
3        2023-01-01              5       233
4        2023-01-01              6        78
...             ...            ...       ...
5067     2024-02-22             12         1
5068     2024-02-22             14         1
5069     2024-02-22             16         1
5070     2024-02-23             13         1
5071     2024-02-25             12         1

[5072 rows x 3 columns]>
       MUNICIPIO_RES     CANTIDAD
count    5072.000000  5072.000000
mean        9.713328    23.033320
std        14.623354    30.053195
min         2.000000     1.000000
25%         6.000000     4.000000
50%        10.000000    11.000000
75%        13.000000    30.000000
max       999.000000   270.000000
FECHA_SINTOMAS
MUNICIPIO_RES
CANTIDAD


In [30]:
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([2], 'AZCAPOTZALCO')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([3], 'COYOACAN')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([4], 'CUAJIMALPA')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([5], 'GUSTAVOAMADERO')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([6], 'IZTACALCO')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([7], 'IZTAPALAPA')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([8], 'N/A')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([9], 'N/A')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([10], 'ALVAROOBREGON')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([11], 'TLAHUAC')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([12], 'TLALPAN')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([13], 'N/A')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([14], 'BENITOJUAREZ')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([15], 'CUAHUTEMOC')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([16], 'MIGUELHIDALGO')
covid_data_daily['MUNICIPIO_RES'] = covid_data_daily['MUNICIPIO_RES'].replace([17], 'VENUSTIANOCARRANZA')

In [31]:
print(covid_data_daily.head)

print(covid_data_daily.describe())

covid_data_daily['FECHA_SINTOMAS'] = pd.to_datetime(covid_data_daily['FECHA_SINTOMAS'], infer_datetime_format=True)
covid_data_daily["CO"]= np.nan
covid_data_daily["O3"]= np.nan
covid_data_daily["PM25"]= np.nan

print(covid_data_daily.head())

<bound method NDFrame.head of      FECHA_SINTOMAS   MUNICIPIO_RES  CANTIDAD
0        2023-01-01    AZCAPOTZALCO        82
1        2023-01-01        COYOACAN       102
2        2023-01-01      CUAJIMALPA        13
3        2023-01-01  GUSTAVOAMADERO       233
4        2023-01-01       IZTACALCO        78
...             ...             ...       ...
5067     2024-02-22         TLALPAN         1
5068     2024-02-22    BENITOJUAREZ         1
5069     2024-02-22   MIGUELHIDALGO         1
5070     2024-02-23             N/A         1
5071     2024-02-25         TLALPAN         1

[5072 rows x 3 columns]>
          CANTIDAD
count  5072.000000
mean     23.033320
std      30.053195
min       1.000000
25%       4.000000
50%      11.000000
75%      30.000000
max     270.000000


In [52]:
urlpm25 = "/content/Maximos_PM2-5_2020_-_2024.csv"
urlco = "/content/Alcaldias_CO.csv"
urlo3 ="/content/Maximos_O3_2020_-_2024.csv"

PM25_data = pd.read_csv(urlpm25)
co_data = pd.read_csv(urlco)
o3_data = pd.read_csv(urlo3)
#PM25_data = PM25_data.rename(columns={'IZTAPA': 'IZTAPALAPA'})

print(PM25_data.describe())

#for col in PM25_data.columns:
    #print(col)

PM25_data['Fecha'] = pd.to_datetime(PM25_data['Fecha'], infer_datetime_format=True)
co_data['Fecha'] = pd.to_datetime(co_data['Fecha'], infer_datetime_format=True)
o3_data['Fecha'] = pd.to_datetime(o3_data['Fecha'], infer_datetime_format=True)

print(PM25_data.head())
print(co_data.head())
print(o3_data.head())


          TLALPAN  BENITOJUAREZ  AZCAPOTZALCO   COYOACAN_1  CUAJIMALPA_1  \
count  269.000000   1389.000000   1069.000000  1366.000000           0.0   
mean    31.650558     35.779698     40.105706    31.554173           NaN   
std     12.275013     14.353920     14.995038    12.763199           NaN   
min      6.000000      6.000000      7.000000     5.000000           NaN   
25%     24.000000     26.000000     30.000000    23.000000           NaN   
50%     31.000000     35.000000     38.000000    30.000000           NaN   
75%     37.000000     43.000000     48.000000    38.000000           NaN   
max     89.000000    144.000000    139.000000   118.000000           NaN   

       GUSTAVOAMADERO  CUAUHTEMOC  IZTACALCO  VENUSTIANOCARRANZA  \
count      633.000000  138.000000        0.0         1005.000000   
mean        41.816746   40.644928        NaN           41.175124   
std         17.584639   13.775468        NaN           15.680423   
min          8.000000   10.000000        Na

<ipython-input-52-f5d4b416a4c2>:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  PM25_data['Fecha'] = pd.to_datetime(PM25_data['Fecha'], infer_datetime_format=True)
<ipython-input-52-f5d4b416a4c2>:16: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  co_data['Fecha'] = pd.to_datetime(co_data['Fecha'], infer_datetime_format=True)
<ipython-input-52-f5d4b416a4c2>:17: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  o3_data['Fecha'] = pd.to_datetime(o3_data['Fecha'], infer_datetime_format=True)


In [54]:

for ind in PM25_data.index:
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="AZCAPOTZALCO"), ['PM25']] = PM25_data['AZCAPOTZALCO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="COYOACAN"), ['PM25']] = PM25_data['COYOACAN'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="CUAJIMALPA"), ['PM25']] = PM25_data['CUAJIMALPA'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="GUSTAVOAMADERO"), ['PM25']] = PM25_data['GUSTAVOAMADERO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="IZTACALCO"), ['PM25']] = PM25_data['IZTACALCO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="IZTAPALAPA"), ['PM25']] = PM25_data['IZTAPALAPA'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="ALVAROOBREGON"), ['PM25']] = PM25_data['ALVAROOBREGON'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="TLAHUAC"), ['PM25']] = PM25_data['TLAHUAC'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="TLALPAN"), ['PM25']] = PM25_data['TLALPAN'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="BENITOJUAREZ"), ['PM25']] = PM25_data['BENITOJUAREZ'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="CUAHUTEMOC"), ['PM25']] = PM25_data['CUAUHTEMOC'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="MIGUELHIDALGO"), ['PM25']] = PM25_data['MIGUELHIDALGO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="VENUSTIANOCARRANZA"), ['PM25']] = PM25_data['VENUSTIANOCARRANZA'][ind]


print(covid_data_daily.head())

  FECHA_SINTOMAS   MUNICIPIO_RES  CANTIDAD    CO    O3   PM25
0     2023-01-01    AZCAPOTZALCO        82  1.72  59.0  139.0
1     2023-01-01        COYOACAN       102  0.72  59.0  111.0
2     2023-01-01      CUAJIMALPA        13  0.46  62.0    NaN
3     2023-01-01  GUSTAVOAMADERO       233   NaN  56.0  225.0
4     2023-01-01       IZTACALCO        78  1.50  60.0    NaN


In [50]:
for ind in o3_data.index:
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="AZCAPOTZALCO"), ['O3']] = o3_data['AZCAPOTZALCO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="COYOACAN"), ['O3']] = o3_data['COYOACAN'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="CUAJIMALPA"), ['O3']] = o3_data['CUAJIMALPA'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="GUSTAVOAMADERO"), ['O3']] = o3_data['GUSTAVOAMADERO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="IZTACALCO"), ['O3']] = o3_data['IZTACALCO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="IZTAPALAPA"), ['O3']] = o3_data['IZTAPALAPA'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="ALVAROOBREGON"), ['O3']] = o3_data['ALVAROOBREGON'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="TLAHUAC"), ['O3']] = o3_data['TLAHUAC'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="TLALPAN"), ['O3']] = o3_data['TLALPAN'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="BENITOJUAREZ"), ['O3']] = o3_data['BENITOJUAREZ'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="CUAHUTEMOC"), ['O3']] = o3_data['CUAUHTEMOC'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="MIGUELHIDALGO"), ['O3']] = o3_data['MIGUELHIDALGO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="VENUSTIANOCARRANZA"), ['O3']] = o3_data['VENUSTIANOCARRANZA'][ind]


print(covid_data_daily.head())

  FECHA_SINTOMAS   MUNICIPIO_RES  CANTIDAD  CO    O3  PM25
0     2023-01-01    AZCAPOTZALCO        82 NaN  59.0  1.72
1     2023-01-01        COYOACAN       102 NaN  59.0  0.72
2     2023-01-01      CUAJIMALPA        13 NaN  62.0  0.46
3     2023-01-01  GUSTAVOAMADERO       233 NaN  56.0   NaN
4     2023-01-01       IZTACALCO        78 NaN  60.0  1.50


In [51]:
for ind in co_data.index:
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="AZCAPOTZALCO"), ['CO']] = co_data['AZCAPOTZALCO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="COYOACAN"), ['CO']] = co_data['COYOACAN'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="CUAJIMALPA"), ['CO']] = co_data['CUAJIMALPA'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="GUSTAVOAMADERO"), ['CO']] = co_data['GUSTAVOAMADERO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="IZTACALCO"), ['CO']] = co_data['IZTACALCO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="IZTAPALAPA"), ['CO']] = co_data['IZTAPALAPA'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="ALVAROOBREGON"), ['CO']] = co_data['ALVAROOBREGON'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="TLAHUAC"), ['CO']] = co_data['TLAHUAC'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="TLALPAN"), ['CO']] = co_data['TLALPAN'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="BENITOJUAREZ"), ['CO']] = co_data['BENITOJUAREZ'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="CUAHUTEMOC"), ['CO']] = co_data['CUAHUTEMOC'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="MIGUELHIDALGO"), ['CO']] = co_data['MIGUELHIDALGO'][ind]
  covid_data_daily.loc[(covid_data_daily.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_daily.MUNICIPIO_RES =="VENUSTIANOCARRANZA"), ['CO']] = co_data['VENUSTIANOCARRANZA'][ind]


print(covid_data_daily.head())

  FECHA_SINTOMAS   MUNICIPIO_RES  CANTIDAD    CO    O3  PM25
0     2023-01-01    AZCAPOTZALCO        82  1.72  59.0  1.72
1     2023-01-01        COYOACAN       102  0.72  59.0  0.72
2     2023-01-01      CUAJIMALPA        13  0.46  62.0  0.46
3     2023-01-01  GUSTAVOAMADERO       233   NaN  56.0   NaN
4     2023-01-01       IZTACALCO        78  1.50  60.0  1.50


In [59]:
#Importamos las bibliotecas que vamos a utilizar
import math
import random
import pandas as pd
import numpy as np

#Datasets
from sklearn.datasets import load_iris

#Model selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, KFold

#Metrics
from sklearn.metrics import accuracy_score

#Classfier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
#Metrics
from sklearn.metrics import r2_score
from sklearn import svm
# Separate features and target variable

X = covid_data_daily[['MUNICIPIO_RES','CO','O3','PM25']]
y = covid_data_daily['CANTIDAD']

# Split the data into training and testing sets using stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X.describe()

covid_data_daily.dtypes

FECHA_SINTOMAS    datetime64[ns]
MUNICIPIO_RES             object
CANTIDAD                   int64
CO                       float64
O3                       float64
PM25                     float64
dtype: object

In [60]:

# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create column transformer to apply transformations to numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [61]:
# each of these models will take a turn as the second Pipeline step
rg1 = DecisionTreeRegressor()
rg2 = svm.SVR()

In [62]:
# create the Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('regressor', rg1)])

In [63]:

# create the parameter dictionary for rg1
params1 = {}
#params1['classifier__alpha'] = [1.0, 2.0]
params1['regressor__criterion'] = ['squared_error','absolute_error','friedman_mse', 'poisson']
params1['regressor__max_depth'] = [None, 5, 10, 15, 20]
params1['regressor__min_samples_split'] = [2, 5, 10]
params1['regressor__min_samples_leaf']= [ 1, 2, 4,8]
params1['regressor'] = [rg1]

# create the parameter dictionary for rg2
params2 = {}
params2['regressor__kernel']= ['linear', 'rbf', 'poly']
params2['regressor__C'] = [0.1,1,10]
params2['regressor__gamma']= ['scale', 'auto']
params2['regressor__degree']= [2,3,4]
params2['regressor'] = [rg2]

# create the parameter dictionary for clf2
#params2 = {}
#params2['regressor__n_neighbors'] =  [3, 5, 7]
#params2['regressor__weights'] = ['uniform', 'distance']
#params2['regressor'] = [rg2]


In [64]:
# create a list of parameter dictionaries
params = [params1, params2]

In [65]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params,scoring='neg_mean_squared_error', error_score='raise')
grid.fit(X_train, y_train)
grid.best_params_
print(grid.best_params_)

{'regressor': DecisionTreeRegressor(criterion='poisson', max_depth=5, min_samples_leaf=2,
                      min_samples_split=10), 'regressor__criterion': 'poisson', 'regressor__max_depth': 5, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 10}


In [66]:
# Access the best model (best estimator) directly
best_model = grid.best_estimator_
best_model_name = best_model.named_steps['regressor'].__class__.__name__
# Now you can use the best_model for further analysis or evaluation
best_predictions = best_model.predict(X_test)
best_accuracy = r2_score(y_test, best_predictions)

print(f"Best Model Test Accuracy: {best_accuracy}")

print(f"Best Model: {best_model_name}")
print(f"Best Model parameters: {best_model['regressor'].get_params()}")


Best Model Test Accuracy: 0.14330580226499623
Best Model: DecisionTreeRegressor
Best Model parameters: {'ccp_alpha': 0.0, 'criterion': 'poisson', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
